# Create harmonized sgRNA guide annotation file for use with the CRISPR pipeline (2025)
This notebook describes the creation of a unified annotation file from the guide annotation files provided by the Hon, Huangfu, and Gersbach labs, according to the specification described in: https://github.com/pinellolab/CRISPR_Pipeline/blob/main/example_data/guide_metadata.tsv 

# Install libraries and set paths

In [1]:
#%pip install pandas
#%pip install matplotlib
#%pip install numpy
#%pip install seaborn
#%pip install biomart

In [2]:
# Imports
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

In [3]:
# Paths: TODO update if necessary
#local_path = "/cellar/users/aklie/data/datasets/tf_perturb_seq/ref/"
#local_path = "C:/Users/seg95/Documents/tf_perturb_seq/"
local_path = "/hpc/group/gersbachlab/seg95/tf_perturb_seq/ref/"
#local_path = "D:/tf_perturb_seq/"

# Import merged guide reference file, along with guide index file

In [4]:
# Merged guide ref file
merged_guide_file = pd.read_csv(local_path + "outer_merged_file.csv")
print(merged_guide_file.head())

merged_guide_file_poolabcd = pd.read_csv(local_path + "outer_merged_file_poolabcd.csv")
merged_guide_file_poolf = pd.read_csv(local_path + "outer_merged_file_poolf.csv")

                         id_hon           protospacer       type  \
0    FOXN1_-_26833391.23-P1P2-1  GCACAGGACGGCCGAGCTGA  targeting   
1     EN2_-_155251011.23-P1P2-1  GCTCCGTGTGCGCCGCGGGA  targeting   
2  BCLAF1_-_136610510.23-P1P2-2  GCTCCGTTGCAACCACACAG  targeting   
3      KLF6_-_3827130.23-P1P2-2  GCTGGAGGATCGATCGGCGG  targeting   
4     ELF1_+_41593362.23-P1P2-2  GTGAGCTGATAAACAGAGGG  targeting   

  intended_target_name_hon    reverse_compliment genomic_element  \
0                    FOXN1  TCAGCTCGGCCGTCCTGTGC        promoter   
1                      EN2  TCCCGCGGCGCACACGGAGC        promoter   
2                   BCLAF1  CTGTGTGGTTGCAACGGAGC        promoter   
3                     KLF6  CCGCCGATCGATCCTCCAGC        promoter   
4                     ELF1  CCCTCTGTTTATCAGCTCAC        promoter   

                    id_gersbach intended_target_name_gersbach id_huangfu  \
0    FOXN1_-_26833391.23-P1P2-1                         FOXN1    FOXN1_1   
1     EN2_-_155251011.23-P1P2-

In [5]:
# sgRNA index files
sgrna_index_poolabcd = pd.read_csv(local_path + "sgRNA_index_v0.csv", sep = "\t")
sgrna_index_poolf = pd.read_csv(local_path + "igvf_poolF_annotation.csv", sep = "\t")

sgrna_index_dacc_annot = pd.read_csv(local_path + "sgRNA_index_dacc_annot_reference.csv", sep = "\t")
print(len(set(sgrna_index_dacc_annot['protospacer']).intersection(set(merged_guide_file_poolabcd['protospacer']))))

def adjust_index_file(sgrna_index, name_sgrna_seq = 'sgRNA_seq', add_leading_G = True):
    if(name_sgrna_seq == "sgRNA_seq"):
        sgrna_index['strand'] = sgrna_index['target_loc'].str.extract(r'\((\+|\-)\)')
        sgrna_index['oligo'] = sgrna_index['oligo'].str.upper()
    else:
        sgrna_index['oligo_sequence'] = sgrna_index['oligo_sequence'].str.upper()
    sgrna_index[name_sgrna_seq] = sgrna_index[name_sgrna_seq].str.upper()
    # Adjust the index file to add leading Gs if needed
    if(add_leading_G):
        sgrna_index[name_sgrna_seq] = 'G' + sgrna_index[name_sgrna_seq]
    return sgrna_index

sgrna_index_poolabcd = adjust_index_file(sgrna_index_poolabcd)
sgrna_index_poolf = adjust_index_file(sgrna_index_poolf, name_sgrna_seq= 'protospacer', add_leading_G = False)

sgrna_index_dacc_annot['protospacer'] = sgrna_index_dacc_annot['protospacer'].str.upper()
sgrna_index_poolf['protospacer'] = sgrna_index_poolf['protospacer'].str.upper()
#sgrna_index_dacc_annot['protospacer'] = [s[1:] if len(s) > 0 else s for s in sgrna_index_dacc_annot['protospacer']]
#sgrna_index_dacc_annot['reverse_compliment'] = sgrna_index_dacc_annot['reverse_compliment'].str.rstrip('C')

13188


In [6]:
# Add a reverse compliment if needed
def reverse_compliment(sequence):
    complement = {'A': 'T', 'T': 'A', 'C': 'G', 'G': 'C'}
    return "".join(complement.get(base, base) for base in reversed(sequence.upper()))

sgrna_index_poolabcd['reverse_compliment'] = sgrna_index_poolabcd['sgRNA_seq'].apply(reverse_compliment)
sgrna_index_poolf.rename(columns={"antisense_sequence": "reverse_compliment"})

print("Index:")
print(sgrna_index_poolabcd.head())
print(sgrna_index_poolf.head())
print("Annot:")
print(sgrna_index_dacc_annot.head())

Index:
                   target_loc              element_seq     target source  \
0  chr17:36948966-36948984(+)  chr17:36948966-36949088  AATF_P1P2     TF   
1  chr17:36949026-36949044(+)  chr17:36948966-36949088  AATF_P1P2     TF   
2  chr17:36949013-36949031(+)  chr17:36948966-36949088  AATF_P1P2     TF   
3  chr17:36949070-36949088(-)  chr17:36948966-36949088  AATF_P1P2     TF   
4  chr17:36949031-36949049(+)  chr17:36948966-36949088  AATF_P1P2     TF   

              sgRNA_seq                                              oligo  \
0  GAGTGGCCGGTCCAGAGCTG  GTGGAAAGGACGAAACACCGAGTGGCCGGTCCAGAGCTGGTTTAAG...   
1  GGGATCAAGGCGAGAGGATC  GTGGAAAGGACGAAACACCGGGATCAAGGCGAGAGGATCGTTTAAG...   
2  GGAGTCGGGGAATCGGATCA  GTGGAAAGGACGAAACACCGGAGTCGGGGAATCGGATCAGTTTAAG...   
3  GAAATGTGCGGCCCAACCCC  GTGGAAAGGACGAAACACCGAAATGTGCGGCCCAACCCCGTTTAAG...   
4  GAAGGCGAGAGGATCCGGCA  GTGGAAAGGACGAAACACCGAAGGCGAGAGGATCCGGCAGTTTAAG...   

  gene_target chr_target  chr_start_target  chr_end_target chr_elem

In [7]:
sgrna_index_dacc_annot["protospacer_upper"] = sgrna_index_dacc_annot["protospacer"].str.upper() 

print(len(set(sgrna_index_poolabcd['sgRNA_seq']).intersection(sgrna_index_dacc_annot['protospacer_upper'])))

13470


In [8]:
# Merge pool A-D index and DACC files into one; pool F file has sufficient info for matching
sgrna_index_merged = pd.merge(
    sgrna_index_dacc_annot,
    sgrna_index_poolabcd,
    left_on=['protospacer_upper', 'reverse_compliment'],
    right_on=['sgRNA_seq', 'reverse_compliment'],
    how="outer"
)
print(sgrna_index_merged.head())
print(sgrna_index_merged.shape)

               protospacer_ID           protospacer intended_target_name  \
0                     OR5K2-2  GAAAAAATTGTAGAGGAATA                OR5K2   
1    SP1_+_53773993.23-P1P2-1  GAAAAACGCGGACGCTGACG                  SP1   
2    SP8_-_20826141.23-P1P2-2  GAAAAAGATCCTCTGAGAGG                  SP8   
3    FOXN3_-_89883583.23-P2-1  GAAAAAGGCGACACATGACC                FOXN3   
4  ZNF85_+_21106076.23-P1P2-1  GAAAACAAGACCTAGAGCTC                ZNF85   

        type genomic_element    reverse_compliment     protospacer_upper  \
0  targeting        promoter  TATTCCTCTACAATTTTTTC  GAAAAAATTGTAGAGGAATA   
1  targeting        promoter  CGTCAGCGTCCGCGTTTTTC  GAAAAACGCGGACGCTGACG   
2  targeting        promoter  CCTCTCAGAGGATCTTTTTC  GAAAAAGATCCTCTGAGAGG   
3  targeting        promoter  GGTCATGTGTCGCCTTTTTC  GAAAAAGGCGACACATGACC   
4  targeting        promoter  GAGCTCTAGGTCTTGTTTTC  GAAAACAAGACCTAGAGCTC   

                   target_loc              element_seq      target  ...  \
0          

# Add positive and negative controls and non-targeting gRNAs, if not already present

In [9]:
neg_controls = pd.read_csv(local_path + "negative_controls.tsv", sep = "\t")
pos_controls = pd.read_csv(local_path + "positive_controls.tsv", sep = "\t")
non_targeting = pd.read_csv(local_path + "non_targeting.tsv", sep = "\t")

print(non_targeting.head())
print(pos_controls.head())
print(neg_controls.head())

            Unnamed: 0 Photospacer (same for all 3 sets)
0  non-targeting_00642              GGAGTTAAGGCCTCGTCTAG
1  non-targeting_00718              GTCCCAGGCTCTCCACTATG
2  non-targeting_03631              GGACGCGTCTGCAAGAACGT
3  non-targeting_03705              GGGCATGGACCCGCGGCACG
4  non-targeting_01469              GCGTCCGAGGTACTGAATAA
           Gene Photospacer (represent 10 times)  \
0   CD81 strong             GGAGAGCGAGCGCGCAACGG   
1     CD81 weak             GGAGAGCCAGCGCGCAACGG   
2  CD151 strong             GCCGGACTCGGACGCGTGGT   
3    CD151 weak             GCCGCTCGGCCGAGCTGTCG   
4   CD55 strong             GCTGCGACTCGGCGGAGTCC   

                                           Reference  
0  Horlbeck et al. 2016 "Compact and highly activ...  
1  Jost et al. 2020 "Titrating gene expression us...  
2  Horlbeck et al. 2016 "Compact and highly activ...  
3  Horlbeck et al. 2016 "Compact and highly activ...  
4  Horlbeck et al. 2016 "Compact and highly activ...  
     Gene      

In [10]:
print(len(set(sgrna_index_merged['protospacer_upper']).intersection(set(non_targeting['Photospacer (same for all 3 sets)']))))
print(len(set(sgrna_index_merged['protospacer_upper']).intersection(set(pos_controls['Photospacer (represent 10 times)']))))  
cols = [c for c in neg_controls.columns if c.startswith('Photospacer')]
neg_spacers = pd.concat([neg_controls[c] for c in cols]).dropna().astype(str)

len(set(sgrna_index_merged['protospacer_upper']).intersection(set(neg_spacers)))

600
19


577

# Reformat to resemble input to the CRISPR pipeline

In [11]:
# Import example file for the CRISPR pipeline
example_crispr_file = pd.read_csv(local_path + "crispr_annot_sample.tsv", sep = "\t")

In [12]:
example_crispr_file.head()

,guide_id,spacer,targeting,type,guide_chr,guide_start,guide_end,strand,pam,intended_target_name,intended_target_chr,intended_target_start,intended_target_end
0,AFF4_sg1,CCAGCGGACGGGGCGGGGAC,True,targeting,chr5,132299282.0,132299302.0,-,NGG,AFF4,chr5,132875395.0,132963634.0
1,AFF4_sg2,CCGCCAGCGGACGGGGCGGC,True,targeting,chr5,132299282.0,132299302.0,-,NGG,AFF4,chr5,132875395.0,132963634.0
2,AFF4_sg3,CGTCCGCTGGCGGCGGCGAC,True,targeting,chr5,132299252.0,132299272.0,-,NGG,AFF4,chr5,132875395.0,132963634.0
3,AFF4_sg4,CTGCGTCAGTCACAGCCCTC,True,targeting,chr5,132299279.0,132299299.0,-,NGG,AFF4,chr5,132875395.0,132963634.0
4,AFF4_sg5,GCGGACGGGGCGGGGATCCC,True,targeting,chr5,132299279.0,132299299.0,-,NGG,AFF4,chr5,132875395.0,132963634.0


In [13]:
# Keep only necessary columns and reorder them to match 
def prune_and_rename_cols(merged_guide_file, is_pool_f=False):
    # start from full table so you don't drop rows prematurely
    df = merged_guide_file.copy()

    if is_pool_f:
        df["guide_id"] = (
            df.get("id_gersbach").combine_first(df.get("id_engreitz"))
        )
        df["intended_target_name"] = (
            df.get("intended_target_name_gersbach")
            .combine_first(df.get("intended_target_name_engreitz"))
        )
    else:
        df["guide_id"] = (
            df.get("id_hon")
            .combine_first(df.get("id_gersbach"))
            .combine_first(df.get("id_engreitz"))
            .combine_first(df.get("id_huangfu"))
        )
        df["intended_target_name"] = (
            df.get("intended_target_name_hon")
            .combine_first(df.get("intended_target_name_gersbach"))
            .combine_first(df.get("intended_target_name_engreitz"))
            .combine_first(df.get("intended_target_name_huangfu"))
        )

    # Fallbacks for control / non‑targeting rows.
    df["guide_id"] = df["guide_id"].fillna(df.get("id", df.get("protospacer")))
    df["intended_target_name"] = df["intended_target_name"].fillna(df.get("type"))

    # Rename after all adjustments
    if "protospacer" in df.columns:
        df = df.rename(columns={"protospacer": "spacer"})

    keep_cols = [c for c in ["guide_id", "spacer", "type", "intended_target_name", "reverse_compliment"] if c in df.columns]
    ref_clean_sub = df[keep_cols].copy()

    print(f"Retained rows: {ref_clean_sub.shape[0]}")
    return ref_clean_sub

# Call function
ref_clean_sub = prune_and_rename_cols(merged_guide_file)
ref_clean_sub_poolabcd = prune_and_rename_cols(merged_guide_file_poolabcd)
ref_clean_sub_poolf = prune_and_rename_cols(merged_guide_file_poolf, is_pool_f=True)

Retained rows: 19956
Retained rows: 17364
Retained rows: 2592


In [14]:
# Add 'targeting' column; if type == targeting, set to True, otherwise False
def check_targeting(value):
    if(value == "targeting"):
        return True
    else:
        return False

def add_targeting_col(ref_clean_sub):
    ref_clean_sub['targeting'] = ref_clean_sub['type'].apply(check_targeting)
    order = ['guide_id', 'spacer', 'targeting', 'type', 'intended_target_name']
    ref_clean_sub = ref_clean_sub[order]
    print(ref_clean_sub.head())
    return ref_clean_sub

ref_clean_sub = add_targeting_col(ref_clean_sub)
ref_clean_sub_poolabcd = add_targeting_col(ref_clean_sub_poolabcd)
ref_clean_sub_poolf = add_targeting_col(ref_clean_sub_poolf)

                       guide_id                spacer  targeting       type  \
0    FOXN1_-_26833391.23-P1P2-1  GCACAGGACGGCCGAGCTGA       True  targeting   
1     EN2_-_155251011.23-P1P2-1  GCTCCGTGTGCGCCGCGGGA       True  targeting   
2  BCLAF1_-_136610510.23-P1P2-2  GCTCCGTTGCAACCACACAG       True  targeting   
3      KLF6_-_3827130.23-P1P2-2  GCTGGAGGATCGATCGGCGG       True  targeting   
4     ELF1_+_41593362.23-P1P2-2  GTGAGCTGATAAACAGAGGG       True  targeting   

  intended_target_name  
0                FOXN1  
1                  EN2  
2               BCLAF1  
3                 KLF6  
4                 ELF1  
                     guide_id                spacer  targeting  \
0  TFEC_-_115670779.23-P1P2-1  GCATATGCACCATGCCAGAA       True   
1  NR2C1_-_95467292.23-P1P2-2  GGATGTGGGATCGAGATTCA       True   
2   NANOG_+_7942459.23-P1P2-2  GTTTTTCCATTATAACTTGG       True   
3                     OR8B3-5  GTTTTTGTCTTCAAAAATCT      False   
4  ZNF48_+_30406782.23-P1P2-1  GCTCCGCGCCAAGC

In [15]:
# Add PAM
def add_pam(ref_clean_sub):
    ref_clean_sub['pam'] = 'NGG'
    print(ref_clean_sub.head())
    print(ref_clean_sub.shape)
    return ref_clean_sub

ref_clean_sub = add_pam(ref_clean_sub)
ref_clean_sub_poolabcd = add_pam(ref_clean_sub_poolabcd)
ref_clean_sub_poolf = add_pam(ref_clean_sub_poolf)

                       guide_id                spacer  targeting       type  \
0    FOXN1_-_26833391.23-P1P2-1  GCACAGGACGGCCGAGCTGA       True  targeting   
1     EN2_-_155251011.23-P1P2-1  GCTCCGTGTGCGCCGCGGGA       True  targeting   
2  BCLAF1_-_136610510.23-P1P2-2  GCTCCGTTGCAACCACACAG       True  targeting   
3      KLF6_-_3827130.23-P1P2-2  GCTGGAGGATCGATCGGCGG       True  targeting   
4     ELF1_+_41593362.23-P1P2-2  GTGAGCTGATAAACAGAGGG       True  targeting   

  intended_target_name  pam  
0                FOXN1  NGG  
1                  EN2  NGG  
2               BCLAF1  NGG  
3                 KLF6  NGG  
4                 ELF1  NGG  
(19956, 6)
                     guide_id                spacer  targeting  \
0  TFEC_-_115670779.23-P1P2-1  GCATATGCACCATGCCAGAA       True   
1  NR2C1_-_95467292.23-P1P2-2  GGATGTGGGATCGAGATTCA       True   
2   NANOG_+_7942459.23-P1P2-2  GTTTTTCCATTATAACTTGG       True   
3                     OR8B3-5  GTTTTTGTCTTCAAAAATCT      False   
4  Z

In [16]:
print(sgrna_index_merged.head())
print(len(set(sgrna_index_merged['protospacer_upper']).intersection(set(ref_clean_sub_poolabcd['spacer']))))
print(len(set(sgrna_index_poolf['protospacer']).intersection(set(ref_clean_sub_poolf['spacer']))))  

               protospacer_ID           protospacer intended_target_name  \
0                     OR5K2-2  GAAAAAATTGTAGAGGAATA                OR5K2   
1    SP1_+_53773993.23-P1P2-1  GAAAAACGCGGACGCTGACG                  SP1   
2    SP8_-_20826141.23-P1P2-2  GAAAAAGATCCTCTGAGAGG                  SP8   
3    FOXN3_-_89883583.23-P2-1  GAAAAAGGCGACACATGACC                FOXN3   
4  ZNF85_+_21106076.23-P1P2-1  GAAAACAAGACCTAGAGCTC                ZNF85   

        type genomic_element    reverse_compliment     protospacer_upper  \
0  targeting        promoter  TATTCCTCTACAATTTTTTC  GAAAAAATTGTAGAGGAATA   
1  targeting        promoter  CGTCAGCGTCCGCGTTTTTC  GAAAAACGCGGACGCTGACG   
2  targeting        promoter  CCTCTCAGAGGATCTTTTTC  GAAAAAGATCCTCTGAGAGG   
3  targeting        promoter  GGTCATGTGTCGCCTTTTTC  GAAAAAGGCGACACATGACC   
4  targeting        promoter  GAGCTCTAGGTCTTGTTTTC  GAAAACAAGACCTAGAGCTC   

                   target_loc              element_seq      target  ...  \
0          

In [17]:
# Check for repeated spacer sequences in index file
print(sgrna_index_merged['protospacer_upper'].value_counts().loc[lambda x: x > 1])

protospacer_upper
GCAGCCACGCGAGAGTAGAA    3
GGACCGCGGCCGAGCGAACC    2
GTGCTGGGAGGCGGTTTCCG    2
GGGACCTGCGGGAAATCGGG    2
GTGGGGAGGAAGCGGTTCTA    2
                       ..
GGAGGGGCTACGGTGACCAG    2
GGTCTCCGCTCTGATGCCTG    2
GCGCTCTGATGCCTGAGGAA    2
GCGCTGCGGTGGAGCCACCG    2
GGCGACAGAAGCCTGGGTAC    2
Name: count, Length: 92, dtype: int64


In [18]:
# Remove multiple mappings from sgrna_index_merged
def deduplicate_index_file(df):
    def chrom_rank(chrom):
        if pd.isna(chrom):
            return 100
        if isinstance(chrom, str) and chrom.startswith("chr"):
            c = chrom[3:]
            if c.isdigit():
                return int(c)
            elif c == "X":
                return 23
            elif c == "Y":
                return 24
        return 100  # fallback
    
    df = df.copy()
    
    # Rank and sorting
    df["chrom_rank"] = df["chr_target"].map(chrom_rank)
    df["sort_key"] = (
        df["chrom_rank"].fillna(100) * 1e12 +
        df["chr_start_target"].fillna(0) * 1e6 +
        (df["chr_end_target"].fillna(0) - df["chr_start_target"].fillna(0))
    )
    
    # Group by spacer sequence
    grouped = df.groupby("protospacer_upper", group_keys=False)
    
    # Keep only groups where all key columns are the same across rows
    key_cols = [
        "chr_target", "chr_start_target", "chr_end_target",
        "chr_element", "chr_start_element", "chr_end_element"
    ]
    
    def is_consistent(group):
        return all(group[col].nunique(dropna=False) == 1 for col in key_cols)
    
    consistent_df = grouped.filter(is_consistent)
    
    # Deduplicate remaining consistent rows by keeping best ranked
    deduped_df = (
        consistent_df.sort_values("sort_key")
                     .drop_duplicates(subset="protospacer_upper", keep="first")
                     .drop(columns=["chrom_rank", "sort_key"])
    )
    
    return deduped_df

# Apply deduplication before merging
sgrna_index_merged = deduplicate_index_file(sgrna_index_merged)

In [19]:
# Add the 'guide_chr', 'guide_start', and 'guide_end' values, which are given as 'chr_target', 'chr_start_target', 'chr_end_target', and 'strand'
def add_guide_coords(ref_clean_sub, sgrna_index_merged):
    ref_clean_sub = pd.merge(
        ref_clean_sub,
        sgrna_index_merged[['protospacer_upper', 'chr_target', 'chr_start_target', 'chr_end_target', 'strand']],
        left_on='spacer',
        right_on='protospacer_upper',
        how='left'
    )
    # Remove protospacer_upper column
    ref_clean_sub = ref_clean_sub.drop(columns=['protospacer_upper'])
    # Rename intended guide names
    ref_clean_sub.rename(columns={'chr_target': 'guide_chr', 
                                  'chr_start_target': 'guide_start',
                                  'chr_end_target': 'guide_end'},
                                  inplace=True)


    print(ref_clean_sub.head())
    return ref_clean_sub


ref_clean_sub_poolabcd = add_guide_coords(ref_clean_sub_poolabcd, sgrna_index_merged)
print(ref_clean_sub_poolabcd.head())

# Columns are already correctly labeled for pool F
ref_clean_sub_poolf = pd.merge(
    ref_clean_sub_poolf,
    sgrna_index_poolf[['protospacer', 'guide_chr', 'guide_start', 'guide_end', 'strand']],
    left_on='spacer',
    right_on='protospacer',
    how='left'
)
print(ref_clean_sub_poolf.head())

                     guide_id                spacer  targeting  \
0  TFEC_-_115670779.23-P1P2-1  GCATATGCACCATGCCAGAA       True   
1  NR2C1_-_95467292.23-P1P2-2  GGATGTGGGATCGAGATTCA       True   
2   NANOG_+_7942459.23-P1P2-2  GTTTTTCCATTATAACTTGG       True   
3                     OR8B3-5  GTTTTTGTCTTCAAAAATCT      False   
4  ZNF48_+_30406782.23-P1P2-1  GCTCCGCGCCAAGCCGGGAG       True   

               type intended_target_name  pam guide_chr  guide_start  \
0         targeting                 TFEC  NGG      chr7  116030705.0   
1         targeting                NR2C1  NGG     chr12   95073493.0   
2         targeting                NANOG  NGG     chr12    7789912.0   
3  negative_control                OR8B3  NGG       NaN          NaN   
4         targeting                ZNF48  NGG     chr16   30395465.0   

     guide_end strand  
0  116030723.0      +  
1   95073511.0      +  
2    7789930.0      +  
3          NaN    NaN  
4   30395483.0      -  
                     guide

In [20]:
# Add the intended_target_chr/intended_target_start/intended_target_end values, which are given as 'chr_element', 'chr_start_element', 'chr_end_element'
# Note that this refers to the element being targeted, not the gene itself
def add_element_coords(ref_clean_sub, sgrna_index_merged):
    ref_clean_sub = pd.merge(
        ref_clean_sub,
        sgrna_index_merged[['protospacer_upper', 'chr_element', 'chr_start_element', 'chr_end_element']],
        left_on='spacer',
        right_on='protospacer_upper',
        how='left'
    )
    # Remove protospacer_upper column
    ref_clean_sub = ref_clean_sub.drop(columns=['protospacer_upper'])
    # Rename intended target names
    ref_clean_sub.rename(columns={'chr_element': 'intended_target_chr', 
                                  'chr_start_element': 'intended_target_start',
                                  'chr_end_element': 'intended_target_end'},
                                  inplace=True)
    print(ref_clean_sub.head())
    return ref_clean_sub


ref_clean_sub_poolabcd = add_element_coords(ref_clean_sub_poolabcd, sgrna_index_merged)
ref_clean_sub_poolabcd.head()

# Columns are already correctly labeled for pool F
ref_clean_sub_poolf = pd.merge(
    ref_clean_sub_poolf,
    sgrna_index_poolf[['protospacer', 'intended_target_chr', 'intended_target_start', 'intended_target_end']],
    left_on='spacer',
    right_on='protospacer',
    how='left'
)

                     guide_id                spacer  targeting  \
0  TFEC_-_115670779.23-P1P2-1  GCATATGCACCATGCCAGAA       True   
1  NR2C1_-_95467292.23-P1P2-2  GGATGTGGGATCGAGATTCA       True   
2   NANOG_+_7942459.23-P1P2-2  GTTTTTCCATTATAACTTGG       True   
3                     OR8B3-5  GTTTTTGTCTTCAAAAATCT      False   
4  ZNF48_+_30406782.23-P1P2-1  GCTCCGCGCCAAGCCGGGAG       True   

               type intended_target_name  pam guide_chr  guide_start  \
0         targeting                 TFEC  NGG      chr7  116030705.0   
1         targeting                NR2C1  NGG     chr12   95073493.0   
2         targeting                NANOG  NGG     chr12    7789912.0   
3  negative_control                OR8B3  NGG       NaN          NaN   
4         targeting                ZNF48  NGG     chr16   30395465.0   

     guide_end strand intended_target_chr  intended_target_start  \
0  116030723.0      +                chr7            116030682.0   
1   95073511.0      +             

In [21]:
print(example_crispr_file.head())

# Reorganize columns to match
new_order = ['guide_id', 'spacer', 'targeting', 'type', 'guide_chr', 'guide_start', 'guide_end', 'strand', 'pam', 'intended_target_name', 'intended_target_chr', 'intended_target_start', 'intended_target_end']
ref_clean_sub_poolabcd = ref_clean_sub_poolabcd[new_order].drop_duplicates()
print(ref_clean_sub_poolabcd.head())
ref_clean_sub_poolf = ref_clean_sub_poolf[new_order].drop_duplicates()
print(ref_clean_sub_poolf.head())

controls_in_ref = ref_clean_sub_poolabcd[
    ref_clean_sub_poolabcd['spacer'].isin(non_targeting['Photospacer (same for all 3 sets)'])
    | ref_clean_sub_poolabcd['spacer'].isin(pos_controls['Photospacer (represent 10 times)'])
    | ref_clean_sub_poolabcd['spacer'].isin(neg_spacers)
]
print(len(controls_in_ref))
print(ref_clean_sub_poolabcd['type'].value_counts(dropna=False))
#controls_in_ref.sample(10)

   guide_id                spacer  targeting       type guide_chr  \
0  AFF4_sg1  CCAGCGGACGGGGCGGGGAC       True  targeting      chr5   
1  AFF4_sg2  CCGCCAGCGGACGGGGCGGC       True  targeting      chr5   
2  AFF4_sg3  CGTCCGCTGGCGGCGGCGAC       True  targeting      chr5   
3  AFF4_sg4  CTGCGTCAGTCACAGCCCTC       True  targeting      chr5   
4  AFF4_sg5  GCGGACGGGGCGGGGATCCC       True  targeting      chr5   

   guide_start    guide_end strand  pam intended_target_name  \
0  132299282.0  132299302.0      -  NGG                 AFF4   
1  132299282.0  132299302.0      -  NGG                 AFF4   
2  132299252.0  132299272.0      -  NGG                 AFF4   
3  132299279.0  132299299.0      -  NGG                 AFF4   
4  132299279.0  132299299.0      -  NGG                 AFF4   

  intended_target_chr  intended_target_start  intended_target_end  
0                chr5            132875395.0          132963634.0  
1                chr5            132875395.0          132963634.

In [22]:
# Remove any rows with non-standard chromosomes (i.e. chr1, chr2, chrX, etc., not chrU) by making sure chr is not followed by a letter other than X or Y
control_types = ['non_targeting', 'negative_control', 'positive_control']

# Mask for control and targeting guides
is_control = ref_clean_sub_poolabcd['type'].str.lower().isin(control_types)
is_targeting = ~is_control

# Standard chromosome pattern
standard_chr_pattern = r'^chr(\d+|X|Y)$'

# Split, filter targeting only
controls_df = ref_clean_sub_poolabcd[is_control]
targets_df = ref_clean_sub_poolabcd[is_targeting]

filtered_targets_df = targets_df[
    targets_df['guide_chr'].notna()
    & targets_df['intended_target_chr'].notna()
    & targets_df['guide_chr'].str.match(standard_chr_pattern)
    & targets_df['intended_target_chr'].str.match(standard_chr_pattern)
]

# Recombine targets + controls
ref_clean_sub_poolabcd = pd.concat([filtered_targets_df, controls_df], ignore_index=True)
ref_clean_sub_poolf = pd.concat([ref_clean_sub_poolf, controls_df])

In [23]:
# There are certain examples where a target has the same protospacer sequence but multiple local_target_start and local_target_end
# values, which throws errors with the pipeline. This takes the min/max of those values (dependent on strand) and collapses into a single row

# Collapse groups of targeting guides that have identical metadata but
# multiple start/end coordinates. For negative‑strand entries, we take the
# max(start) / min(end); for positive‑strand or mixed, min(start) / max(end).
# Control or non‑targeting rows are passed through unchanged.
def collapse_grouped_targets(df):

    def collapse_group(subdf):
        # Handle both guide_ and intended_target_ coordinates
        if (subdf["strand"] == "-").all():
            g_start = subdf["guide_start"].max()
            g_end   = subdf["guide_end"].min()
            t_start = subdf["intended_target_start"].max()
            t_end   = subdf["intended_target_end"].min()
        else:
            g_start = subdf["guide_start"].min()
            g_end   = subdf["guide_end"].max()
            t_start = subdf["intended_target_start"].min()
            t_end   = subdf["intended_target_end"].max()

        row = subdf.iloc[0].copy()
        row["guide_start"] = g_start
        row["guide_end"] = g_end
        row["intended_target_start"] = t_start
        row["intended_target_end"] = t_end
        return row

    # Identify control / non-targeting rows (pass through unchanged)
    is_control = (
        df["type"].str.contains("control", case=False, na=False)
        | df["type"].str.contains("non", case=False, na=False)
    )

    controls_df = df[is_control].copy()
    targets_df  = df[~is_control].copy()

    # Exclude all coordinate columns from grouping
    coord_cols = [
        "guide_start", "guide_end",
        "intended_target_start", "intended_target_end"
    ]
    group_cols = [c for c in targets_df.columns if c not in coord_cols]

    collapsed_targets = (
        targets_df
        .groupby(group_cols, dropna=False)
        .apply(collapse_group)
        .reset_index(drop=True)
    )

    combined = pd.concat([collapsed_targets, controls_df], ignore_index=True)
    return combined

ref_clean_sub_poolabcd = collapse_grouped_targets(ref_clean_sub_poolabcd)
ref_clean_sub_poolf    = collapse_grouped_targets(ref_clean_sub_poolf)
print(ref_clean_sub_poolabcd.shape)
print(ref_clean_sub_poolf.shape)

/tmp/ipykernel_2963810/1879577626.py:47: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  targets_df


(14172, 13)
(3862, 13)


/tmp/ipykernel_2963810/1879577626.py:47: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  targets_df


In [24]:
# Also write a version without mostly NA values, duplicate rows
#print(ref_clean_sub_poolabcd.shape)
#ref_clean_sub_poolabcd_clean = ref_clean_sub_poolabcd.dropna(thresh = (len(ref_clean_sub_poolabcd.columns)/2)).drop_duplicates()
#print(ref_clean_sub_poolabcd_clean.shape)
#ref_clean_sub_poolabcd_clean.to_csv(local_path + "harmonized_guide_file_poolabcd_nomissing.csv")

In [25]:
# Interrogate duplicate spacers
duplicate_spacers_poolf = ref_clean_sub_poolf[ref_clean_sub_poolf["spacer"].duplicated(keep=False)]
print(duplicate_spacers_poolf["spacer"].value_counts())

duplicate_spacers_poolf_diff = duplicate_spacers_poolf.loc[:, duplicate_spacers_poolf.nunique() > 1]
print(duplicate_spacers_poolf_diff.head())
print(duplicate_spacers_poolf_diff.shape)

spacer
CGCCGGCGCGCCTGCGAGG    4
TCCTGCGATATCCAGGCGA    4
CTCCTTGCAGCCACCACGG    4
GCTCACGTCATCCCGACCG    4
CGACACTACCAGCTGCTGT    4
CAAATCCTCCTGTCTTTCG    4
CGTGCAAAACCCTGTGCCT    4
CAACTTGCCACTCAAACGC    2
GCCGGAGCTACCGGCAGCC    2
GCTCCGCCGCTCGGCCCCT    2
CACGTAACGGGACCACACA    2
GACGCCCCCGGCCAGGTGA    2
CACTTGCAGGGGCGCGAGG    2
GTCCTTCCCGTCGCCTGCA    2
AGTGAGGACTAACGGGGCA    2
GGAAACCGCCAGACACCAA    2
CACGCCAGACCACGACGGA    2
GCTCCACCCTTTCCGGGCG    2
Name: count, dtype: int64
    guide_id               spacer                guide_chr  guide_start  \
644    DGCR6  CGCCGGCGCGCCTGCGAGG                    chr22   18905981.0   
645    DGCR6  CGCCGGCGCGCCTGCGAGG                    chr22   18905981.0   
646    DGCR6  CGCCGGCGCGCCTGCGAGG  chr22_KI270734v1_random     131252.0   
647    DGCR6  CGCCGGCGCGCCTGCGAGG  chr22_KI270734v1_random     131252.0   
648  DGCR6.2  TCCTGCGATATCCAGGCGA                    chr22   18906057.0   

      guide_end strand intended_target_name      intended_target_c

In [26]:
# Drop alternate contigs if a canonical chr exists (e.g. chr22_KI270731v1_random)
import re
def remove_random_contigs(df):
    df = df.copy()
    keep_rows = []

    canonical_pattern = re.compile(r"^chr(\d+|X|Y)$", re.IGNORECASE)

    for spacer, subdf in df.groupby("spacer", group_keys=False):
        has_main = (
            subdf["guide_chr"].astype(str).str.match(canonical_pattern).any() or
            subdf["intended_target_chr"].astype(str).str.match(canonical_pattern).any()
        )

        if has_main:
            mask = (
                subdf["guide_chr"].astype(str).str.match(canonical_pattern)
                & subdf["intended_target_chr"].astype(str).str.match(canonical_pattern)
            )
            keep_rows.append(subdf[mask])
        else:
            # If no canonical version exists, keep all
            keep_rows.append(subdf)

    cleaned = pd.concat(keep_rows, ignore_index=True)
    return cleaned

before = len(ref_clean_sub_poolf)
ref_clean_sub_poolf = remove_random_contigs(ref_clean_sub_poolf)
after = len(ref_clean_sub_poolf)

print(f"Removed {before - after} '_random' contig rows covered by canonical entries.")

duplicate_spacers_poolf = ref_clean_sub_poolf[ref_clean_sub_poolf["spacer"].duplicated(keep=False)]
print(duplicate_spacers_poolf["spacer"].value_counts())
print(duplicate_spacers_poolf.head(10))

duplicate_spacers_poolf_diff = duplicate_spacers_poolf.loc[:, duplicate_spacers_poolf.nunique() > 1]
print(duplicate_spacers_poolf_diff)
print(duplicate_spacers_poolf_diff.shape)

Removed 21 '_random' contig rows covered by canonical entries.
spacer
AGTGAGGACTAACGGGGCA    2
CAACTTGCCACTCAAACGC    2
CACGCCAGACCACGACGGA    2
CACGTAACGGGACCACACA    2
CACTTGCAGGGGCGCGAGG    2
GACGCCCCCGGCCAGGTGA    2
GCCGGAGCTACCGGCAGCC    2
GCTCCACCCTTTCCGGGCG    2
GCTCCGCCGCTCGGCCCCT    2
GGAAACCGCCAGACACCAA    2
GTCCTTCCCGTCGCCTGCA    2
Name: count, dtype: int64
           guide_id               spacer  targeting       type guide_chr  \
375         NCF1B.6  AGTGAGGACTAACGGGGCA       True  targeting      chr7   
376         NCF1B.6  AGTGAGGACTAACGGGGCA       True  targeting      chr7   
458        GATSL2.2  CAACTTGCCACTCAAACGC       True  targeting      chr7   
459        GATSL2.2  CAACTTGCCACTCAAACGC       True  targeting      chr7   
488       STAG3L2.3  CACGCCAGACCACGACGGA       True  targeting      chr7   
489       STAG3L2.3  CACGCCAGACCACGACGGA       True  targeting      chr7   
494  LOC100101148.2  CACGTAACGGGACCACACA       True  targeting      chr7   
495  LOC100101148.2  

In [41]:
# Manually remove 11 strand mismatches in Pool F to select the most canonical one
canonical_strand_choices = {
    "AGTGAGGACTAACGGGGCA": "+",  # NCF1B.6
    "CAACTTGCCACTCAAACGC": "+",  # GATSL2.3
    "CACGCCAGACCACGACGGA": "-",  # STAG3L2.3
    "CACGTAACGGGACCACACA": "+",  # LOC100101148.2
    "CACTTGCAGGGGCGCGAGG": "+", # LOC541473.2
    "GACGCCCCCGGCCAGGTGA": "+",  # LOC100101148.6
    "GCCGGAGCTACCGGCAGCC": "-",  # GTF2IP1.2
    "GCTCCACCCTTTCCGGGCG": "-",  # STAG3L3.5
    "GCTCCGCCGCTCGGCCCCT": "-",  # GTF2IP1.9
    "GGAAACCGCCAGACACCAA": "-",  # STAG3L2.2
    "GTCCTTCCCGTCGCCTGCA": "+",  # NCF1B
}
mask_keep = pd.Series(True, index=ref_clean_sub_poolf.index)

for spacer, strand in canonical_strand_choices.items():
    # Identify all rows for this spacer
    idx_all = ref_clean_sub_poolf.index[ref_clean_sub_poolf["spacer"] == spacer]
    # Identify those on non-canonical strands
    idx_wrong = ref_clean_sub_poolf.index[
        (ref_clean_sub_poolf["spacer"] == spacer)
        & (ref_clean_sub_poolf["strand"] != strand)
    ]
    # Mark wrong-strand rows for removal
    mask_keep.loc[idx_wrong] = False
    print(f"Keeping {spacer} ({strand}), removing {len(idx_wrong)} opposite-strand rows")

# Apply mask
before = len(ref_clean_sub_poolf)
ref_clean_sub_poolf = ref_clean_sub_poolf.loc[mask_keep].reset_index(drop=True)
after = len(ref_clean_sub_poolf)

print(f"Removed {before - after} strand-mismatched rows.")
print(f"{ref_clean_sub_poolf['spacer'].duplicated().sum()} duplicate spacers remain.")

Keeping AGTGAGGACTAACGGGGCA (+), removing 0 opposite-strand rows
Keeping CAACTTGCCACTCAAACGC (+), removing 0 opposite-strand rows
Keeping CACGCCAGACCACGACGGA (-), removing 0 opposite-strand rows
Keeping CACGTAACGGGACCACACA (+), removing 0 opposite-strand rows
Keeping CACTTGCAGGGGCGCGAGG (+), removing 0 opposite-strand rows
Keeping GACGCCCCCGGCCAGGTGA (+), removing 0 opposite-strand rows
Keeping GCCGGAGCTACCGGCAGCC (-), removing 0 opposite-strand rows
Keeping GCTCCACCCTTTCCGGGCG (-), removing 0 opposite-strand rows
Keeping GCTCCGCCGCTCGGCCCCT (-), removing 0 opposite-strand rows
Keeping GGAAACCGCCAGACACCAA (-), removing 0 opposite-strand rows
Keeping GTCCTTCCCGTCGCCTGCA (+), removing 0 opposite-strand rows
Removed 0 strand-mismatched rows.
0 duplicate spacers remain.


In [28]:
# Create a merged Pool ABCD and Pool F file
ref_clean_sub_poolabcdf = pd.concat(
    [ref_clean_sub_poolabcd, ref_clean_sub_poolf], ignore_index=True
).drop_duplicates(subset=["spacer"], keep="first")
print(ref_clean_sub_poolabcdf.head())
print(ref_clean_sub_poolabcdf.shape)

                    guide_id                spacer  targeting       type  \
0  AATF_-_35306286.23-P1P2-1  GAGTGGCCGGTCCAGAGCTG       True  targeting   
1  AATF_-_35306286.23-P1P2-2  GGGATCAAGGCGAGAGGATC       True  targeting   
2  AATF_-_35306333.23-P1P2-1  GGAGTCGGGGAATCGGATCA       True  targeting   
3  AATF_-_35306333.23-P1P2-2  GAAATGTGCGGCCCAACCCC       True  targeting   
4  AATF_-_35306351.23-P1P2-1  GAAGGCGAGAGGATCCGGCA       True  targeting   

  guide_chr  guide_start   guide_end strand  pam intended_target_name  \
0     chr17   36948966.0  36948984.0      +  NGG                 AATF   
1     chr17   36949026.0  36949044.0      +  NGG                 AATF   
2     chr17   36949013.0  36949031.0      +  NGG                 AATF   
3     chr17   36949070.0  36949088.0      -  NGG                 AATF   
4     chr17   36949031.0  36949049.0      +  NGG                 AATF   

  intended_target_chr  intended_target_start  intended_target_end  
0               chr17             36

In [29]:
# Another check for duplicates in the concatenated file
duplicate_spacers = ref_clean_sub_poolabcdf[ref_clean_sub_poolabcdf.duplicated(subset=['spacer'])]
#print(duplicate_spacers.head())
#print(duplicate_spacers.shape)

# Find the columns that are different between the duplicate spacers
duplicate_spacers_diff = duplicate_spacers.loc[:, duplicate_spacers.nunique() > 1]
#print(duplicate_spacers_diff.head())
#print(duplicate_spacers_diff.shape)

In [30]:
# Fix any Excel-style gene names converted to dates
import re
month_gene_map = {
    "JAN": "JAN",
    "FEB": "FEB",
    "MAR": "MARCH",
    "APR": "APR",
    "MAY": "MAY",
    "JUN": "JUN",
    "JUL": "JUL",
    "AUG": "AUG",
    "SEP": "SEPT",
    "OCT": "OCT",
    "NOV": "NOV",
    "DEC": "DEC"
}

def fix_excel_date_genes(symbol):
    # Convert Excel-mangled gene symbols like 5-SEP to SEPT5
    m = re.match(r"^(\d{1,2})-([A-Z]{3})$", symbol)
    if m and m.group(2) in month_gene_map:
        num, month = m.groups()
        return f"{month_gene_map[month]}{num}"
    return symbol

ref_clean_sub_poolabcdf['intended_target_name'] = ref_clean_sub_poolabcdf['intended_target_name'].apply(fix_excel_date_genes)
ref_clean_sub_poolabcd['intended_target_name'] = ref_clean_sub_poolabcd['intended_target_name'].apply(fix_excel_date_genes)
ref_clean_sub_poolf['intended_target_name'] = ref_clean_sub_poolf['intended_target_name'].apply(fix_excel_date_genes)

print(len(set(ref_clean_sub_poolabcdf['spacer']).intersection(set(non_targeting['Photospacer (same for all 3 sets)']))))
print(len(set(ref_clean_sub_poolabcdf['spacer']).intersection(set(pos_controls['Photospacer (represent 10 times)']))))  
len(set(ref_clean_sub_poolabcdf['spacer']).intersection(set(neg_spacers)))

600
19


577

In [31]:
# Write to file
ref_clean_sub_poolabcd.to_csv(local_path + "harmonized_guide_file_poolabcd.csv")
ref_clean_sub_poolabcd.to_csv(
    local_path + "harmonized_guide_file_poolabcd.tsv",
    sep='\t',
    index=False,
    lineterminator='\n'
)
ref_clean_sub_poolf.to_csv(local_path + "harmonized_guide_file_poolf.csv")
ref_clean_sub_poolf.to_csv(
    local_path + "harmonized_guide_file_poolf.tsv",
    sep='\t',
    index=False,
    lineterminator='\n'
)

In [32]:
ref_clean_sub_poolabcdf.to_csv(local_path + "harmonized_guide_file_poolabcdf.csv")

# Write to tsv file, including header
ref_clean_sub_poolabcdf.to_csv(
    local_path + "harmonized_guide_file_poolabcdf.tsv",
    sep='\t',
    index=False,
    lineterminator='\n'
)

---

In [33]:
#%pip install pybiomart

In [34]:
# Convert intended_target_name to Ensembl ID using pyBiomart
from pybiomart import Dataset

dataset = Dataset(name='hsapiens_gene_ensembl', host='http://www.ensembl.org')

# Fetch mapping
mapping = dataset.query(attributes=['hgnc_symbol', 'ensembl_gene_id', 'external_synonym'])
mapping.columns = ['intended_target_name', 'ensembl_gene_id', 'external_synonym']
mapping.head()

,intended_target_name,ensembl_gene_id,external_synonym
0,MT-TF,ENSG00000210049,MTTF
1,MT-TF,ENSG00000210049,TRNF
2,MT-RNR1,ENSG00000211459,12S
3,MT-RNR1,ENSG00000211459,MOTS-C
4,MT-RNR1,ENSG00000211459,MTRNR1


In [35]:
# Combine HGNC symbol and synonyms into a single mapping dataframe
# Melt external_synonym if it's a comma-separated list
mapping_expanded = mapping.copy()
mapping_expanded['external_synonym'] = mapping_expanded['external_synonym'].fillna('')
mapping_expanded = mapping_expanded.assign(
    synonym_list=mapping_expanded['external_synonym'].str.split(',')
).explode('synonym_list')
mapping_expanded.head()

,intended_target_name,ensembl_gene_id,external_synonym,synonym_list
0,MT-TF,ENSG00000210049,MTTF,MTTF
1,MT-TF,ENSG00000210049,TRNF,TRNF
2,MT-RNR1,ENSG00000211459,12S,12S
3,MT-RNR1,ENSG00000211459,MOTS-C,MOTS-C
4,MT-RNR1,ENSG00000211459,MTRNR1,MTRNR1


In [36]:
# Combine intended_target_name and synonym_list into one lookup table
lookup = pd.concat([
    mapping_expanded[['intended_target_name', 'ensembl_gene_id']].rename(columns={'intended_target_name': 'symbol'}),
    mapping_expanded[['synonym_list', 'ensembl_gene_id']].rename(columns={'synonym_list': 'symbol'})
]).drop_duplicates()
#print(lookup.head())
lookup['symbol'] = lookup['symbol'].apply(lambda x: str(x).upper().replace('-', '').replace('_',''))
print(lookup.head())

    symbol  ensembl_gene_id
0     MTTF  ENSG00000210049
2   MTRNR1  ENSG00000211459
5     MTTV  ENSG00000210077
7   MTRNR2  ENSG00000210082
10   MTTL1  ENSG00000209082


In [37]:
def clean_symbol(s):
    s = str(s).upper()
    # Remove dashes for relaxed matching
    s = s.replace('-', '').replace('_', '')
    return s

In [38]:
# Merge with data frame and replace intended_target_name with Ensembl IDs
def replace_w_ensembl(ref_clean, mapping):
    # Make all symbols uppercase for matching
    ref_clean = ref_clean.copy()
    ref_clean['intended_target_name'] = ref_clean['intended_target_name'].apply(clean_symbol)

    mapping = mapping.copy()
    mapping['symbol'] = mapping['symbol'].apply(clean_symbol)

    # Merge by symbol
    ref_clean = ref_clean.merge(mapping, left_on='intended_target_name',
                                right_on='symbol', how='left')

    # Identify missing mappings
    missing_mask = ref_clean['ensembl_gene_id'].isna()
    num_missing = missing_mask.sum()
    missing_genes = ref_clean.loc[missing_mask, 'intended_target_name'].unique()

    # Print summary
    print(f"Number of rows with missing Ensembl mapping: {num_missing}")
    print(f"Gene symbols with no mapping:\n{missing_genes}")

    # Replace intended_target_name with Ensembl ID where available,
    # otherwise keep the original gene name
    ref_clean['intended_target_name'] = ref_clean.apply(
        lambda row: row['ensembl_gene_id'] if pd.notna(row['ensembl_gene_id']) 
                    else row['intended_target_name'],
        axis=1
    )

    # Drop temp columns if you don’t need them later
    ref_clean.drop(columns=['ensembl_gene_id', 'symbol'], inplace=True, errors='ignore')

    return ref_clean

ref_clean_sub_poolabcdf_ensembl = replace_w_ensembl(ref_clean_sub_poolabcdf, lookup)
ref_clean_sub_poolabcd_ensembl = replace_w_ensembl(ref_clean_sub_poolabcd, lookup)
ref_clean_sub_poolf_ensembl = replace_w_ensembl(ref_clean_sub_poolf, lookup)

Number of rows with missing Ensembl mapping: 739
Gene symbols with no mapping:
['NONTARGETING' 'OR2C36' 'OR56A44' 'OR11H61' 'OR2W15' 'OR9Q15' 'OR1N25'
 'OR9Q11' 'OR2C32' 'OR2H11' 'OR2C33' 'OR56A43' 'OR2W12' 'OR9Q16' 'OR9Q13'
 'OR2C35' 'OR11H65' 'OR9Q12' 'OR2W11' 'OR2C34' 'OR56A42' 'OR2C31'
 'SEPT5GP1BB' 'CTD2574D22' 'LOC100101148' 'XXBACB562F10' '5SEP'
 'LOC101926943' 'GREGOR' 'LOC100133091' 'CTD2515O10' 'LOC388849'
 'LOC541473' 'LOC284865']
Number of rows with missing Ensembl mapping: 621
Gene symbols with no mapping:
['NONTARGETING' 'OR2C36' 'OR56A44' 'OR11H61' 'OR2W15' 'OR9Q15' 'OR1N25'
 'OR9Q11' 'OR2C32' 'OR2H11' 'OR2C33' 'OR56A43' 'OR2W12' 'OR9Q16' 'OR9Q13'
 'OR2C35' 'OR11H65' 'OR9Q12' 'OR2W11' 'OR2C34' 'OR56A42' 'OR2C31']
Number of rows with missing Ensembl mapping: 739
Gene symbols with no mapping:
['SEPT5GP1BB' 'CTD2574D22' 'LOC100101148' 'XXBACB562F10' '5SEP'
 'LOC101926943' 'GREGOR' 'LOC100133091' 'CTD2515O10' 'LOC388849'
 'LOC541473' 'LOC284865' 'NONTARGETING' 'OR9Q11' 'OR2C

In [39]:
# Write to file
ref_clean_sub_poolabcd.to_csv(local_path + "harmonized_guide_file_poolabcd_ensg.csv")
ref_clean_sub_poolabcd.to_csv(
    local_path + "harmonized_guide_file_poolabcd_ensg.tsv",
    sep='\t',
    index=False,
    lineterminator='\n'
)
ref_clean_sub_poolf.to_csv(local_path + "harmonized_guide_file_poolf_ensg.csv")
ref_clean_sub_poolf.to_csv(
    local_path + "harmonized_guide_file_poolf_ensg.tsv",
    sep='\t',
    index=False,
    lineterminator='\n'
)
ref_clean_sub_poolabcdf.to_csv(local_path + "harmonized_guide_file_poolabcdf_ensg.csv")

# Write to tsv file, including header
ref_clean_sub_poolabcdf.to_csv(
    local_path + "harmonized_guide_file_poolabcdf_ensg.tsv",
    sep='\t',
    index=False,
    lineterminator='\n'
)

In [43]:
# Quick unit tests to make sure everything is kosher
def run_integrity_checks(df, pool_label=""):
    print(f"\nRunning integrity checks for {pool_label}")

    # Check for duplicate spacers
    duplicates = df[df["spacer"].duplicated()]
    assert len(duplicates) == 0, f"{len(duplicates)} duplicate spacers found in {pool_label}"

    # Check NA values are np.nan (not 'NA', 'None', or empty strings)
    bad_na = df.isin(["NA", "None", ""]).any().sum()
    assert bad_na == 0, f"{bad_na} non-numeric NA placeholders found in {pool_label}"

    # Check for strange characters in spacer or guide_id
    pattern_ok = re.compile(r"^[ACGTN]+$", re.IGNORECASE)
    bad_spacers = df[~df["spacer"].astype(str).str.match(pattern_ok)]
    assert len(bad_spacers) == 0, f"Unexpected characters in {len(bad_spacers)} spacers"

    # Confirm control guides are present
    control_types = ["non_targeting", "positive_control", "negative_control"]
    found_controls = {ct: (df["type"].str.lower() == ct).sum() for ct in control_types}
    missing_controls = [ct for ct, count in found_controls.items() if count == 0]
    assert not missing_controls, f"Missing control types: {missing_controls}"

    # Confirm coordinate columns are numeric or np.nan
    coord_cols = ["guide_start", "guide_end", "intended_target_start", "intended_target_end"]
    for col in coord_cols:
        if col in df.columns:
            bad_coords = df[col].dropna().apply(lambda x: isinstance(x, (int, float)))
            assert bad_coords.all(), f"Non-numeric entries in {col}"

    # Confirm chromosome format (allow chr1–22, chrX/Y, and *_random)
    chr_cols = ["guide_chr", "intended_target_chr"]
    chr_pattern = re.compile(r"^chr(\d+|X|Y)(_.*_random)?$", re.IGNORECASE)
    for col in chr_cols:
        if col in df.columns:
            # Only check non-NaN entries
            bad_chr = df[col].dropna().astype(str).apply(lambda x: not chr_pattern.match(x))
            assert not bad_chr.any(), f"Invalid chromosome names in {col}"

    print(f"All checks passed for {pool_label}")

run_integrity_checks(ref_clean_sub_poolabcd, "ABCD")
run_integrity_checks(ref_clean_sub_poolf, "F")
run_integrity_checks(ref_clean_sub_poolabcdf, "ABCDF")


Running integrity checks for ABCD
All checks passed for ABCD

Running integrity checks for F
All checks passed for F

Running integrity checks for ABCDF
All checks passed for ABCDF
